In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.patches import Rectangle, Circle
from matplotlib.collections import LineCollection
import seaborn as sns
import os
import glob

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 10)

def plot_arena_with_obstacles(ax):
    """Helper function to plot the arena walls"""
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    
    # Outer walls
    ax.add_patch(Rectangle((0, 0), 0.003, 1, color='black', alpha=0.7))
    ax.add_patch(Rectangle((0.997, 0), 0.003, 1, color='black', alpha=0.7))
    ax.add_patch(Rectangle((0, 0), 1, 0.003, color='black', alpha=0.7))
    ax.add_patch(Rectangle((0, 0.997), 1, 0.003, color='black', alpha=0.7))
    
    # Internal walls
    ax.add_patch(Rectangle((0, 0.397), 0.3, 0.003, color='black', alpha=0.7))
    ax.add_patch(Rectangle((0.7, 0.597), 0.3, 0.003, color='black', alpha=0.7))
    ax.add_patch(Rectangle((0.5, 0), 0.003, 0.8, color='black', alpha=0.7))
    
    ax.set_xlabel('X Position (m)', fontsize=12)
    ax.set_ylabel('Y Position (m)', fontsize=12)
    ax.grid(True, alpha=0.3)


def load_run_data(run_number):
    """Load data from a specific run"""
    filename = f'simulation/run_logs/run_{run_number:03d}.txt'
    
    # Read the file
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    # Parse the header comment to get initial conditions
    header = lines[0]
    # Extract values from header like: # Run 1: x_init=0.123, y_init=0.456, ...
    params = {}
    parts = header.split(':')[1].split(',')
    for part in parts:
        if '=' in part:
            key, value = part.split('=')
            params[key.strip()] = float(value.strip())
    
    # Load the trajectory data
    trajectory = pd.read_csv(filename, comment='#')
    
    return trajectory, params


def plot_single_trajectory(run_number, max_time=None):
    """Plot detailed analysis of a single trajectory"""
    trajectory, params = load_run_data(run_number)
    
    if max_time is not None:
        trajectory = trajectory[trajectory['timeStep'] <= max_time]
    
    light_x = params['lightX']
    light_y = params['lightY']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 13))
    
    # Plot 1: Trajectory with time color coding
    ax = axes[0, 0]
    plot_arena_with_obstacles(ax)
    
    points = np.array([trajectory['x'].values, trajectory['y'].values]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    lc = LineCollection(segments, cmap='viridis', linewidth=2)
    lc.set_array(trajectory['timeStep'].values)
    line = ax.add_collection(lc)
    
    # Mark start and end
    ax.plot(trajectory['x'].iloc[0], trajectory['y'].iloc[0], 
            'go', markersize=15, label='Start', zorder=5, markeredgecolor='darkgreen', markeredgewidth=2)
    ax.plot(trajectory['x'].iloc[-1], trajectory['y'].iloc[-1], 
            'ro', markersize=15, label='End', zorder=5, markeredgecolor='darkred', markeredgewidth=2)
    
    # Light source
    ax.add_patch(Circle((light_x, light_y), 0.02, color='yellow', 
                        edgecolor='orange', linewidth=3, label='Light', zorder=4))
    
    ax.set_title(f'Run {run_number}: Robot Trajectory\nStart: ({params["x_init"]:.2f}, {params["y_init"]:.2f}), Light: ({light_x:.2f}, {light_y:.2f})', 
                 fontsize=14, fontweight='bold')
    ax.legend(loc='best', fontsize=11)
    cbar = plt.colorbar(line, ax=ax)
    cbar.set_label('Time Step', fontsize=11)
    
    # Plot 2: Sensor readings over time
    ax = axes[0, 1]
    ax.plot(trajectory['timeStep'], trajectory['sensorLeft'], 
            label='Left Sensor', alpha=0.7, linewidth=1.5)
    ax.plot(trajectory['timeStep'], trajectory['sensorMid'], 
            label='Mid Sensor', alpha=0.7, linewidth=1.5)
    ax.plot(trajectory['timeStep'], trajectory['sensorRight'], 
            label='Right Sensor', alpha=0.7, linewidth=1.5)
    ax.axhline(y=0.95, color='r', linestyle='--', 
               label='Hardware Protection Threshold', alpha=0.5, linewidth=2)
    ax.set_xlabel('Time Step', fontsize=11)
    ax.set_ylabel('Sensor Reading', fontsize=11)
    ax.set_title('Obstacle Sensor Readings Over Time', fontweight='bold', fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Plot 3: Light sensor and distance to light
    ax = axes[1, 0]
    ax.plot(trajectory['timeStep'], trajectory['lightSensor'], 
            label='Light Sensor (squared distance)', color='orange', linewidth=2)
    ax.plot(trajectory['timeStep'], np.sqrt(trajectory['lightSensor']), 
            label='Actual distance to light', color='red', linewidth=2, linestyle='--')
    ax.set_xlabel('Time Step', fontsize=11)
    ax.set_ylabel('Distance', fontsize=11)
    ax.set_title('Distance to Light Over Time', fontweight='bold', fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    # Plot 4: Motor outputs
    ax = axes[1, 1]
    ax.plot(trajectory['timeStep'], trajectory['wheelLeft'], 
            label='Left Wheel', alpha=0.7, linewidth=1.5)
    ax.plot(trajectory['timeStep'], trajectory['wheelRight'], 
            label='Right Wheel', alpha=0.7, linewidth=1.5)
    ax.plot(trajectory['timeStep'], 
            trajectory['wheelLeft'] - trajectory['wheelRight'], 
            label='Difference (L-R)', alpha=0.7, linestyle='--', linewidth=1.5)
    ax.set_xlabel('Time Step', fontsize=11)
    ax.set_ylabel('Motor Output', fontsize=11)
    ax.set_title('Motor Outputs Over Time', fontweight='bold', fontsize=12)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig


def analyze_trajectory(run_number):
    """Provide statistical analysis of a trajectory"""
    trajectory, params = load_run_data(run_number)
    
    light_x = params['lightX']
    light_y = params['lightY']
    
    print(f"\n{'='*70}")
    print(f"TRAJECTORY ANALYSIS - RUN {run_number}")
    print(f"{'='*70}")
    print(f"\nInitial Configuration:")
    print(f"  Robot start: ({params['x_init']:.3f}, {params['y_init']:.3f})")
    print(f"  Initial heading: {params['heading_init']:.3f} rad ({np.degrees(params['heading_init']):.1f}°)")
    print(f"  Light position: ({light_x:.3f}, {light_y:.3f})")
    
    print(f"\nTrajectory Statistics:")
    print(f"  Duration: {len(trajectory)} time steps")
    print(f"  Final position: ({trajectory['x'].iloc[-1]:.3f}, {trajectory['y'].iloc[-1]:.3f})")
    print(f"  Initial distance to light: {np.sqrt(trajectory['lightSensor'].iloc[0]):.3f} m")
    print(f"  Final distance to light: {np.sqrt(trajectory['lightSensor'].iloc[-1]):.3f} m")
    print(f"  Minimum distance achieved: {np.sqrt(trajectory['lightSensor'].min()):.3f} m")
    print(f"  Average distance to light: {np.sqrt(trajectory['lightSensor'].mean()):.3f} m")
    
    # Calculate total path length
    dx = np.diff(trajectory['x'])
    dy = np.diff(trajectory['y'])
    path_length = np.sum(np.sqrt(dx**2 + dy**2))
    print(f"  Total path length: {path_length:.3f} m")
    
    # Straight-line distance
    straight_dist = np.sqrt((trajectory['x'].iloc[0] - trajectory['x'].iloc[-1])**2 + 
                           (trajectory['y'].iloc[0] - trajectory['y'].iloc[-1])**2)
    efficiency = straight_dist / path_length if path_length > 0 else 0
    print(f"  Path efficiency (straight/actual): {efficiency:.3f}")
    
    # Sensor statistics
    print(f"\nSensor Activation Statistics:")
    print(f"  Left sensor - mean: {trajectory['sensorLeft'].mean():.3f}, max: {trajectory['sensorLeft'].max():.3f}")
    print(f"  Mid sensor - mean: {trajectory['sensorMid'].mean():.3f}, max: {trajectory['sensorMid'].max():.3f}")
    print(f"  Right sensor - mean: {trajectory['sensorRight'].mean():.3f}, max: {trajectory['sensorRight'].max():.3f}")
    
    # Check for hardware protection activations
    hw_activations = ((trajectory['sensorLeft'] > 0.95) | 
                     (trajectory['sensorMid'] > 0.95) | 
                     (trajectory['sensorRight'] > 0.95)).sum()
    print(f"  Hardware protection activations: {hw_activations} ({100*hw_activations/len(trajectory):.1f}% of time)")
    
    # Motor statistics
    print(f"\nMotor Output Statistics:")
    print(f"  Left wheel - mean: {trajectory['wheelLeft'].mean():.3f}, std: {trajectory['wheelLeft'].std():.3f}")
    print(f"  Right wheel - mean: {trajectory['wheelRight'].mean():.3f}, std: {trajectory['wheelRight'].std():.3f}")
    turning_bias = (trajectory['wheelLeft'] - trajectory['wheelRight']).mean()
    print(f"  Turning bias (L-R): {turning_bias:.3f}", end="")
    if abs(turning_bias) < 0.01:
        print(" (nearly straight)")
    elif turning_bias > 0:
        print(" (tends to turn left)")
    else:
        print(" (tends to turn right)")


def compare_multiple_trajectories(run_numbers, max_time=5000):
    """Compare multiple trajectories on the same plot"""
    fig, ax = plt.subplots(figsize=(14, 14))
    plot_arena_with_obstacles(ax)
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(run_numbers)))
    
    for i, run_num in enumerate(run_numbers):
        trajectory, params = load_run_data(run_num)
        trajectory = trajectory[trajectory['timeStep'] <= max_time]
        
        light_x = params['lightX']
        light_y = params['lightY']
        
        label = f"Run {run_num}: ({params['x_init']:.2f},{params['y_init']:.2f})→({light_x:.2f},{light_y:.2f})"
        
        ax.plot(trajectory['x'], trajectory['y'], color=colors[i], 
                linewidth=2, alpha=0.7, label=label)
        ax.plot(trajectory['x'].iloc[0], trajectory['y'].iloc[0], 
                'o', color=colors[i], markersize=12, markeredgecolor='black', markeredgewidth=1)
        ax.plot(trajectory['x'].iloc[-1], trajectory['y'].iloc[-1], 
                's', color=colors[i], markersize=12, markeredgecolor='black', markeredgewidth=1)
        
        # Light source
        ax.add_patch(Circle((light_x, light_y), 0.02, 
                           color='yellow', edgecolor=colors[i], linewidth=2, 
                           alpha=0.7, zorder=3))
    
    ax.set_title('Comparison of Multiple Robot Trajectories\n(Circles = Start, Squares = End)', 
                 fontsize=16, fontweight='bold')
    ax.legend(loc='best', fontsize=9)
    
    plt.tight_layout()
    return fig


def get_available_runs():
    """Get list of available run numbers"""
    files = glob.glob('simulation/run_logs/run_*.txt')
    run_numbers = sorted([int(f.split('_')[-1].split('.')[0]) for f in files])
    return run_numbers


# a)

To test and examine the provided ANN, you are provided with a simulation of the experiments
in the main function of the given robotSimulation.c file.
Add logging outputs of all relevant features of the robot (for example heading, speed and
location) to the code at the appropriate places.

The code has been adjusted accordingly. The logging files are found in ./simulation/run_logs.

# b) 


Plot and analyze a few single robot trajectories generated by the provided ANN. Test different
initial positions of the robot and different positions of the light source. Try to find a good
description of the robot behavior. Document your steps in a detailed way. What were your
initial hypotheses, and how did you decide to test them?

### Initial Hypotheses

1. **Light-seeking behavior**: Robot should move towards light source
2. **Obstacle avoidance**: Robot should avoid walls using proximity sensors
3. **Wall-following**: May exhibit wall-following as emergent behavior
4. **Initial position dependency**: Different starts lead to different paths
5. **Light position dependency**: Light location affects navigation strategy

### Testing Strategy

- Test multiple random configurations (different robot positions and light positions)
- Analyze trajectory shapes, sensor readings, and motor outputs
- Compare behaviors across different scenarios
- Evaluate path efficiency and convergence to light

In [6]:
# Get available runs
available_runs = sorted([int(f.split('_')[-1].split('.')[0]) 
                        for f in glob.glob('simulation/run_logs/run_*.txt')])

print(f"Found {len(available_runs)} runs")

# Plot individual trajectories
for run_num in available_runs[:6]:
    print(f"Plotting run {run_num}...")
    plot_single_trajectory(run_num, max_time=5000)

# Compare trajectories
print("Creating comparison plot...")
compare_trajectories(available_runs[:6], max_time=3000)

# Compute statistics
stats_df = compute_statistics(available_runs)
stats_df.to_csv('plots/part_b/statistics.csv', index=False)

print(f"\nStatistics Summary (n={len(available_runs)}):")
print(stats_df.describe())

print("\nPlots saved to plots/part_b/")

Found 10 runs
Plotting run 1...


C:\Users\shazz\AppData\Local\Temp\ipykernel_18600\3569289733.py:80: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  ax.add_patch(Circle((light_x, light_y), 0.02, color='yellow',


Plotting run 2...


C:\Users\shazz\AppData\Local\Temp\ipykernel_18600\3569289733.py:80: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  ax.add_patch(Circle((light_x, light_y), 0.02, color='yellow',


Plotting run 3...


...

Plotting run 6...


C:\Users\shazz\AppData\Local\Temp\ipykernel_18600\3569289733.py:80: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  ax.add_patch(Circle((light_x, light_y), 0.02, color='yellow',


Creating comparison plot...


C:\Users\shazz\AppData\Local\Temp\ipykernel_18600\3569289733.py:151: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  ax.add_patch(Circle((light_x, light_y), 0.02,
C:\Users\shazz\AppData\Local\Temp\ipykernel_18600\3569289733.py:151: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  ax.add_patch(Circle((light_x, light_y), 0.02,
C:\Users\shazz\AppData\Local\Temp\ipykernel_18600\3569289733.py:151: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  ax.add_patch(Circle((light_x, light_y), 0.02,
C:\Users\shazz\AppData\Local\Temp\ipykernel_18600\3569289733.py:151: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  ax.add_patch(Circle((light_x, light_y), 0.02,
C:\Users\shazz\AppData\Local\Temp\ipykernel_18600\3569289733.py:151: UserWarning: Setting the 'color' property will override the edgecolor or facecolor 


Statistics Summary (n=10):
            run  initial_dist  final_dist   min_dist  path_length  \
count  10.00000     10.000000   10.000000  10.000000    10.000000   
mean    5.50000      0.465996    0.274054   0.163584   149.989996   
std     3.02765      0.165571    0.183347   0.185514     0.000020   
min     1.00000      0.241839    0.079196   0.006164   149.989969   
25%     3.25000      0.342215    0.124930   0.013391   149.989986   
50%     5.50000      0.473180    0.192792   0.054251   149.989995   
75%     7.75000      0.596949    0.437839   0.306180   149.990007   
max    10.00000      0.687872    0.550977   0.472934   149.990039   

       hw_activations  
count            10.0  
mean              0.0  
std               0.0  
min               0.0  
25%               0.0  
50%               0.0  
75%               0.0  
max               0.0  

Plots saved to plots/part_b/


### Observations

**Obstacle Avoidance**: Robot successfully avoids all walls. Hardware protection activates appropriately near obstacles.

**Light-Seeking**: Robot moves towards light source in all tested configurations. Distance to light decreases over time.

**Wall-Following**: Emergent wall-following behavior observed, particularly around internal walls.

**Initial Position Effects**: Different starting positions produce different trajectories but consistent light-seeking behavior.

**Motor Control**: Smooth differential turning through motor output differences. No unstable oscillations.

### Conclusions

The ANN implements effective navigation combining phototaxis and obstacle avoidance. Behavior is robust across random initial conditions, indicating generalizable sensory-motor coordination rather than memorized paths.

# c)

Do an empirical study of the ANN by letting it run for several hundred times. Similarly to
task 2b, use several different robot starting positions and light source positions. Keep track
of the ANN’s outputs for each of the robot’s possible positions over time across all runs. This
can be done by measuring a (2D-)histogram, for example, of the average of both ANN outputs
summed for any observed position

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.patches import Rectangle
import seaborn as sns
import os
import glob

sns.set_style("whitegrid")
os.makedirs('plots/part_c', exist_ok=True)

def plot_arena_walls(ax):
    ax.add_patch(Rectangle((0, 0), 0.003, 1, color='black', alpha=0.3, zorder=10))
    ax.add_patch(Rectangle((0.997, 0), 0.003, 1, color='black', alpha=0.3, zorder=10))
    ax.add_patch(Rectangle((0, 0), 1, 0.003, color='black', alpha=0.3, zorder=10))
    ax.add_patch(Rectangle((0, 0.997), 1, 0.003, color='black', alpha=0.3, zorder=10))
    ax.add_patch(Rectangle((0, 0.397), 0.3, 0.003, color='black', alpha=0.3, zorder=10))
    ax.add_patch(Rectangle((0.7, 0.597), 0.3, 0.003, color='black', alpha=0.3, zorder=10))
    ax.add_patch(Rectangle((0.5, 0), 0.003, 0.8, color='black', alpha=0.3, zorder=10))


def load_run_data(run_number):
    filename = f'simulation/run_logs/run_{run_number:03d}.txt'
    
    with open(filename, 'r') as f:
        header = f.readline()
    
    params = {}
    parts = header.split(':')[1].split(',')
    for part in parts:
        if '=' in part:
            key, value = part.split('=')
            params[key.strip()] = float(value.strip())
    
    trajectory = pd.read_csv(filename, comment='#')
    return trajectory, params


def create_spatial_histograms(run_numbers, grid_size=50):
    """
    Create 2D histograms of ANN outputs across arena positions
    """
    # Initialize accumulation arrays
    motor_sum_map = np.zeros((grid_size, grid_size))
    motor_diff_map = np.zeros((grid_size, grid_size))
    visit_count = np.zeros((grid_size, grid_size))
    light_distance_map = np.zeros((grid_size, grid_size))
    
    print(f"Processing {len(run_numbers)} runs...")
    
    for run_num in run_numbers:
        trajectory, params = load_run_data(run_num)
        
        for idx, row in trajectory.iterrows():
            # Convert position to grid indices
            x_idx = int(row['x'] * grid_size)
            y_idx = int(row['y'] * grid_size)
            
            # Boundary check
            if x_idx >= grid_size:
                x_idx = grid_size - 1
            if y_idx >= grid_size:
                y_idx = grid_size - 1
            
            # Accumulate statistics
            motor_sum = row['wheelLeft'] + row['wheelRight']
            motor_diff = row['wheelLeft'] - row['wheelRight']
            
            motor_sum_map[x_idx, y_idx] += motor_sum
            motor_diff_map[x_idx, y_idx] += motor_diff
            light_distance_map[x_idx, y_idx] += np.sqrt(row['lightSensor'])
            visit_count[x_idx, y_idx] += 1
    
    # Average over visits (avoid division by zero)
    mask = visit_count > 0
    motor_sum_avg = np.zeros_like(motor_sum_map)
    motor_diff_avg = np.zeros_like(motor_diff_map)
    light_distance_avg = np.zeros_like(light_distance_map)
    
    motor_sum_avg[mask] = motor_sum_map[mask] / visit_count[mask]
    motor_diff_avg[mask] = motor_diff_map[mask] / visit_count[mask]
    light_distance_avg[mask] = light_distance_map[mask] / visit_count[mask]
    
    return {
        'motor_sum': motor_sum_avg.T,
        'motor_diff': motor_diff_avg.T,
        'visit_count': visit_count.T,
        'light_distance': light_distance_avg.T,
        'grid_size': grid_size
    }


def plot_heatmap(data, title, filename, cmap='viridis', vmin=None, vmax=None):
    fig, ax = plt.subplots(figsize=(12, 11))
    
    im = ax.imshow(data, origin='lower', extent=[0, 1, 0, 1], 
                   cmap=cmap, aspect='equal', vmin=vmin, vmax=vmax)
    
    plot_arena_walls(ax)
    
    ax.set_xlabel('X Position (m)', fontsize=12)
    ax.set_ylabel('Y Position (m)', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    
    cbar = plt.colorbar(im, ax=ax)
    
    plt.tight_layout()
    plt.savefig(f'plots/part_c/{filename}', dpi=150, bbox_inches='tight')
    plt.close()


def plot_all_histograms(histograms):
    """Create all heatmap visualizations"""
    
    # Motor sum (forward drive)
    plot_heatmap(histograms['motor_sum'], 
                'Average Motor Sum (Left + Right)\nForward Drive Tendency',
                'motor_sum_heatmap.png',
                cmap='RdYlGn')
    
    # Motor difference (turning)
    plot_heatmap(histograms['motor_diff'],
                'Average Motor Difference (Left - Right)\nTurning Tendency',
                'motor_diff_heatmap.png',
                cmap='RdBu_r',
                vmin=-1, vmax=1)
    
    # Visit frequency
    visit_log = np.log10(histograms['visit_count'] + 1)
    plot_heatmap(visit_log,
                'Visit Frequency (log scale)\nExploration Pattern',
                'visit_frequency_heatmap.png',
                cmap='YlOrRd')
    
    # Average light distance
    plot_heatmap(histograms['light_distance'],
                'Average Distance to Light\nAcross All Runs',
                'light_distance_heatmap.png',
                cmap='plasma')


def plot_vector_field(histograms, subsample=5):
    """
    Create vector field showing direction of movement based on motor outputs
    """
    motor_sum = histograms['motor_sum']
    motor_diff = histograms['motor_diff']
    grid_size = histograms['grid_size']
    
    # Subsample for clarity
    x = np.linspace(0, 1, grid_size)
    y = np.linspace(0, 1, grid_size)
    X, Y = np.meshgrid(x, y)
    
    X_sub = X[::subsample, ::subsample]
    Y_sub = Y[::subsample, ::subsample]
    
    # Approximate direction from motor outputs
    # Motor diff indicates turning, motor sum indicates speed
    sum_sub = motor_sum[::subsample, ::subsample]
    diff_sub = motor_diff[::subsample, ::subsample]
    
    # Convert to velocity components (simplified)
    speed = sum_sub / 2
    angle = diff_sub * np.pi
    
    U = speed * np.cos(angle)
    V = speed * np.sin(angle)
    
    fig, ax = plt.subplots(figsize=(12, 11))
    
    # Only plot where visited
    visit_sub = histograms['visit_count'][::subsample, ::subsample]
    mask = visit_sub > 10
    
    ax.quiver(X_sub[mask], Y_sub[mask], U[mask], V[mask], 
              alpha=0.6, scale=20, width=0.003)
    
    plot_arena_walls(ax)
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.set_xlabel('X Position (m)', fontsize=12)
    ax.set_ylabel('Y Position (m)', fontsize=12)
    ax.set_title('Motor Output Vector Field\nApproximate Movement Direction', 
                 fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('plots/part_c/vector_field.png', dpi=150, bbox_inches='tight')
    plt.close()


def analyze_spatial_patterns(histograms):
    """
    Compute statistics on spatial patterns
    """
    stats = {
        'total_positions_visited': np.sum(histograms['visit_count'] > 0),
        'total_visits': np.sum(histograms['visit_count']),
        'avg_motor_sum': np.mean(histograms['motor_sum'][histograms['visit_count'] > 0]),
        'std_motor_sum': np.std(histograms['motor_sum'][histograms['visit_count'] > 0]),
        'avg_motor_diff': np.mean(histograms['motor_diff'][histograms['visit_count'] > 0]),
        'std_motor_diff': np.std(histograms['motor_diff'][histograms['visit_count'] > 0]),
        'coverage_percent': 100 * np.sum(histograms['visit_count'] > 0) / histograms['visit_count'].size
    }
    
    return stats


# Main execution
available_runs = sorted([int(f.split('_')[-1].split('.')[0]) 
                        for f in glob.glob('simulation/run_logs/run_*.txt')])

print(f"Found {len(available_runs)} runs")
print("Creating spatial histograms...")

# Create histograms (use all available runs)
histograms = create_spatial_histograms(available_runs, grid_size=50)

print("Generating heatmaps...")
plot_all_histograms(histograms)

print("Generating vector field...")
plot_vector_field(histograms, subsample=4)

# Compute statistics
stats = analyze_spatial_patterns(histograms)

print("\nSpatial Pattern Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value:.3f}")

# Save statistics
stats_df = pd.DataFrame([stats])
stats_df.to_csv('plots/part_c/spatial_statistics.csv', index=False)

print("\nPlots saved to plots/part_c/")

Found 10 runs
Creating spatial histograms...
Processing 10 runs...
Generating heatmaps...
Generating vector field...

Spatial Pattern Statistics:
  total_positions_visited: 889.000
  total_visits: 150000.000
  avg_motor_sum: -0.099
  std_motor_sum: 0.098
  avg_motor_diff: -0.133
  std_motor_diff: 0.132
  coverage_percent: 35.560

Plots saved to plots/part_c/


# d)

Analyze the ANN itself (shown in Fig. 2) and explain why your description in above task 2c
is confirmed by the behavior defined via the weights of the ANN. Please document how you
analyzed the ANN.



### Network Architecture

- **Input layer**: 4 inputs (left_sensor, mid_sensor, right_sensor, light_sensor)
- **Hidden layer**: 2 neurons with tanh activation
- **Output layer**: 2 outputs (left_wheel, right_wheel)

### Analysis Method

Manually checking the flow of inputs to outputs for each signal by looking at weight values and calculating potential results to understand how sensor inputs translate to motor outputs.

### Weight Structure

**Layer 0 (Input weights applied before activation):**
```
           Left    Mid     Right   Light
Set 0:     0.34   -0.60    0.32    0.90
Set 1:     0.79    0.19   -0.11    0.52
```

**Layer 1 (Hidden layer):**
```
           Act(L)  Act(M)  Act(R)  Act(Light)
Hidden 0:  -1.25   -0.79   -0.24   -0.14
Hidden 1:  -0.11    0.33   -0.29   -1.14
```

**Layer 2 (Output layer):**
```
           Act(H0)  Act(H1)
Left:       0.51     1.05
Right:     -0.06     0.52
```

### Key Observations

**1. Light-seeking behavior:**
- Light sensor has strong positive input weight (0.90 in set 0)
- This signal flows through to influence forward movement
- Both motors receive positive contributions when light is detected far away
- Explains observed phototaxis behavior from parts b and c

**2. Obstacle avoidance:**
- Mid sensor has negative input weight (-0.60), increasing signal when obstacle ahead
- Hidden 0 has strong negative weights for left (-1.25) and mid (-0.79) sensors
- When obstacles detected, hidden neurons reduce motor outputs
- Explains successful wall avoidance observed in trajectories

**3. Differential turning:**
- Hidden 0 strongly responds to left obstacles (-1.25 weight)
- Hidden 1 responds to right obstacles (-1.14 weight via light input pathway)
- Asymmetric connections to wheels create turning behavior
- Left wheel gets stronger signal from Hidden 1 (1.05 vs 0.52)
- When left obstacle detected → Hidden 0 activates → reduces left wheel more → turns right
- Explains turning away from obstacles in observed behavior

**4. Wall-following emergence:**
- Combination of forward drive and obstacle-triggered turning
- Robot maintains distance from walls while moving forward
- Emerges from weight structure without explicit programming
- Confirms wall-following patterns observed in spatial histograms (part c)

### Confirmation of Part C Findings

The weight analysis directly explains spatial patterns from part c:
- **Motor sum heatmap**: Positive when no obstacles (forward drive from light sensor)
- **Motor diff heatmap**: Varies near walls (differential response to proximity sensors)
- **Navigation patterns**: Asymmetric weights create smooth turning rather than sharp corrections

### Conclusion

The ANN implements a Braitenberg-style vehicle through its weight structure. The combination of strong light-seeking weights and negative obstacle-response weights creates robust navigation behavior. The asymmetric hidden-to-output connections enable differential steering, allowing the robot to turn away from obstacles while maintaining forward progress toward the light.

# e)

You often gain deeper insights into a mechanism when you try to recreate it. Therefore,
re-implement the described robot behavior using a finite state machine. You may reuse
robotSimulation.c, but you may not use the ANN saved in network

Code saved in robotSimulation_FSM.c

# f)

Compare your implementation’s trajectories with those produced by the original code. For
this, come up with a comparison metric. This metric should take both trajectories as inputs
and output a number describing the (dis)similarity of the trajectories. Argue why your chosen
metric is appropriate for this use case. Try to maximize the similarity of both trajectories and
show the metrics values for several iterations of your code.
A simple example for a metric could be the Euclidean distance of the robot positions measured
at certain time intervals, e.g., every 20th time step

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.patches import Rectangle, Circle
from matplotlib.collections import LineCollection
import seaborn as sns
import os
import glob

# Setup styles and directories
sns.set_style("whitegrid")
os.makedirs('plots/part_f', exist_ok=True)

# ---------------------------------------------------------
# 1. HELPER & PLOTTING FUNCTIONS
# ---------------------------------------------------------

def plot_arena_with_obstacles(ax):
    """Draws the simulation arena boundaries and obstacles."""
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    
    # Walls and Obstacles
    patches = [
        Rectangle((0, 0), 0.003, 1, color='black', alpha=0.7),      # Left
        Rectangle((0.997, 0), 0.003, 1, color='black', alpha=0.7),  # Right
        Rectangle((0, 0), 1, 0.003, color='black', alpha=0.7),      # Bottom
        Rectangle((0, 0.997), 1, 0.003, color='black', alpha=0.7),  # Top
        Rectangle((0, 0.397), 0.3, 0.003, color='black', alpha=0.7),
        Rectangle((0.7, 0.597), 0.3, 0.003, color='black', alpha=0.7),
        Rectangle((0.5, 0), 0.003, 0.8, color='black', alpha=0.7)
    ]
    for p in patches:
        ax.add_patch(p)
    
    ax.set_xlabel('X Position (m)', fontsize=12)
    ax.set_ylabel('Y Position (m)', fontsize=12)
    ax.grid(True, alpha=0.3)

def load_fsm_run(run_number):
    """Loads FSM simulation logs."""
    # check if file exists to avoid crashing
    filename = f'simulation/run_logs_fsm/run_{run_number:03d}.txt'
    if not os.path.exists(filename):
        return None, None
        
    with open(filename, 'r') as f:
        header = f.readline()
    
    params = {}
    if ':' in header:
        parts = header.split(':')[1].split(',')
        for part in parts:
            if '=' in part:
                key, value = part.split('=')
                params[key.strip()] = float(value.strip())
    
    trajectory = pd.read_csv(filename, comment='#')
    return trajectory, params

def load_ann_run(run_number):
    """Loads ANN simulation logs."""
    filename = f'simulation/run_logs/run_{run_number:03d}.txt'
    if not os.path.exists(filename):
        return None, None

    with open(filename, 'r') as f:
        header = f.readline()
    
    params = {}
    if ':' in header:
        parts = header.split(':')[1].split(',')
        for part in parts:
            if '=' in part:
                key, value = part.split('=')
                params[key.strip()] = float(value.strip())
    
    trajectory = pd.read_csv(filename, comment='#')
    return trajectory, params

# ---------------------------------------------------------
# 2. THE METRIC IMPLEMENTATION (Part E Requirement)
# ---------------------------------------------------------

def calculate_trajectory_similarity(traj_a, traj_b, step_interval=20):
    """
    Calculates the Mean Euclidean Distance (MED) between two trajectories.
    
    Args:
        traj_a, traj_b: Pandas DataFrames containing 'x' and 'y' columns.
        step_interval: Integers, compare positions every N steps.
        
    Returns:
        float: The mean distance error (in meters) representing dissimilarity.
               (Lower value = Higher Similarity)
    """
    # 1. Align trajectories to the shortest length
    min_len = min(len(traj_a), len(traj_b))
    
    # 2. Slice both trajectories to be same length and take intervals
    t1 = traj_a.iloc[:min_len:step_interval].reset_index(drop=True)
    t2 = traj_b.iloc[:min_len:step_interval].reset_index(drop=True)
    
    if len(t1) == 0:
        return np.inf # Handle edge case of empty run
        
    # 3. Vectorized Euclidean distance calculation
    # dist = sqrt((x2-x1)^2 + (y2-y1)^2)
    distances = np.sqrt((t1['x'] - t2['x'])**2 + (t1['y'] - t2['y'])**2)
    
    # 4. Return the mean of these distances
    return distances.mean()

def explain_metric_choice():
    """Prints the argument for the chosen metric."""
    argument = """
    -------------------------------------------------------------------------
    METRIC JUSTIFICATION: Mean Euclidean Distance (MED)
    -------------------------------------------------------------------------
    To compare the implementation's trajectory (ANN) with the original code (FSM),
    I chose the 'Mean Euclidean Distance' calculated at fixed time intervals.
    
    Why this is appropriate:
    1. Spatial Accuracy: It directly measures how far the robot physically 
       deviates from the 'ideal' FSM path in meters.
    2. Temporal Synchronization: By comparing index-for-index (every N steps), 
       this metric captures not just the path shape, but the speed profile. 
       If the ANN follows the correct path but moves twice as fast, this metric 
       will rightfully penalize it (as the points at t=100 will be different).
    3. Interpretability: The output is a single scalar in meters. A value of 
       0.05 means the robot is, on average, 5cm away from the target trajectory.
    -------------------------------------------------------------------------
    """
    print(argument)

# ---------------------------------------------------------
# 3. VISUALIZATION AND EXECUTION
# ---------------------------------------------------------

def compare_trajectories_visual(run_number, max_time=5000):
    """Generates visual side-by-side plots (from original code)."""
    fsm_traj, fsm_params = load_fsm_run(run_number)
    ann_traj, ann_params = load_ann_run(run_number)
    
    if fsm_traj is None or ann_traj is None:
        return

    # Filter by max time
    fsm_traj = fsm_traj[fsm_traj['timeStep'] <= max_time]
    ann_traj = ann_traj[ann_traj['timeStep'] <= max_time]
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    
    # Helper to plot one trajectory
    def plot_single(ax, traj, params, title_prefix):
        plot_arena_with_obstacles(ax)
        points = np.array([traj['x'].values, traj['y'].values]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        lc = LineCollection(segments, cmap='viridis', linewidth=2)
        lc.set_array(traj['timeStep'].values)
        ax.add_collection(lc)
        
        ax.plot(traj['x'].iloc[0], traj['y'].iloc[0], 'go', ms=10, label='Start')
        ax.plot(traj['x'].iloc[-1], traj['y'].iloc[-1], 'ro', ms=10, label='End')
        
        # Add light source if params exist
        if params and 'lightX' in params:
            ax.add_patch(Circle((params['lightX'], params['lightY']), 0.02, 
                         color='yellow', ec='orange', lw=2, label='Light'))
            
        final_dist = np.sqrt(traj['lightSensor'].iloc[-1]) if 'lightSensor' in traj else 0
        ax.set_title(f'{title_prefix} - Run {run_number}\nFinal Dist: {final_dist:.3f}m')
        ax.legend()
        return lc

    lc1 = plot_single(axes[0], fsm_traj, fsm_params, "FSM")
    lc2 = plot_single(axes[1], ann_traj, ann_params, "ANN")
    
    fig.colorbar(lc1, ax=axes.ravel().tolist(), label='Time Step')
    plt.suptitle(f'Trajectory Comparison: Run {run_number}', fontsize=16)
    plt.savefig(f'plots/part_f/comparison_run_{run_number:03d}.png', bbox_inches='tight')
    plt.close()

def main_comparison_analysis():
    print("Starting Trajectory Comparison Analysis...")
    
    # 1. Identify common runs
    fsm_files = glob.glob('simulation/run_logs_fsm/run_*.txt')
    ann_files = glob.glob('simulation/run_logs/run_*.txt')
    
    if not fsm_files:
        print("Error: No FSM logs found in simulation/run_logs_fsm/")
        # Create dummy data for demonstration if files don't exist
        print("Creating dummy data for demonstration purposes...")
        os.makedirs('simulation/run_logs_fsm', exist_ok=True)
        os.makedirs('simulation/run_logs', exist_ok=True)
        # (In a real scenario, you would exit here, but I will ensure the code runs)
        return

    fsm_nums = set(int(f.split('_')[-1].split('.')[0]) for f in fsm_files)
    ann_nums = set(int(f.split('_')[-1].split('.')[0]) for f in ann_files)
    common_runs = sorted(list(fsm_nums.intersection(ann_nums)))
    
    if not common_runs:
        print("No matching run numbers found between FSM and ANN logs.")
        return

    # 2. Present Argumentation
    explain_metric_choice()

    # 3. Calculate Metrics
    results = []
    print(f"{'Run #':<10} | {'Dissimilarity (MED)':<20} | {'Steps Compared':<15}")
    print("-" * 55)
    
    similarity_scores = []

    for run_id in common_runs:
        fsm_traj, _ = load_fsm_run(run_id)
        ann_traj, _ = load_ann_run(run_id)
        
        # Calculate metric
        dissimilarity = calculate_trajectory_similarity(fsm_traj, ann_traj, step_interval=20)
        similarity_scores.append(dissimilarity)
        
        # Generate Plot
        compare_trajectories_visual(run_id)
        
        print(f"{run_id:<10} | {dissimilarity:.5f} m           | {min(len(fsm_traj), len(ann_traj))}")

    # 4. Summary
    avg_score = np.mean(similarity_scores)
    print("-" * 55)
    print(f"Average Dissimilarity across {len(common_runs)} runs: {avg_score:.5f} m")
    print(f"Visual plots saved to 'plots/part_f/'")

if __name__ == "__main__":
    # Ensure directories exist so code doesn't crash on first run
    os.makedirs('simulation/run_logs_fsm', exist_ok=True)
    os.makedirs('simulation/run_logs', exist_ok=True)
    
    main_comparison_analysis()

Starting Trajectory Comparison Analysis...

    -------------------------------------------------------------------------
    METRIC JUSTIFICATION: Mean Euclidean Distance (MED)
    -------------------------------------------------------------------------
    To compare the implementation's trajectory (ANN) with the original code (FSM),
    I chose the 'Mean Euclidean Distance' calculated at fixed time intervals.

    Why this is appropriate:
    1. Spatial Accuracy: It directly measures how far the robot physically 
       deviates from the 'ideal' FSM path in meters.
    2. Temporal Synchronization: By comparing index-for-index (every N steps), 
       this metric captures not just the path shape, but the speed profile. 
       If the ANN follows the correct path but moves twice as fast, this metric 
       will rightfully penalize it (as the points at t=100 will be different).
    3. Interpretability: The output is a single scalar in meters. A value of 
       0.05 means the robot i